# Gone (randomly?) viral

### Final project in Modelling and Computational Engineering (MOD 510) Dec 2020

Lars B Lukerstuen (248800)

Joakim O Gjermundstad (251365)

# Abstract

# Introdution

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
from functools import reduce

In [583]:
class DiseaseStatus(Enum):  
    SUSCEPTIBLE = 0
    INFECTIOUS = 1
    RECOVERED = 2
    DEAD = 3

class RandomWalkEpidemicSimulator:
    """
    Class used to model the spreading of a contagious disease in a
    population of individuals with a 2D random walk.

    Each walker has a disease status which is represented by an 
    integer Enum. Also, a set of integer (x, y)-coordinates are 
    stored for each walker. The possible coordinates are:

        {0, 1, ..., Lx-1} in the x-direction
        {0, 1, ..., Ly-1} in the y-direction

    It is only possible to move North, South, East, or West. If a 
    walker attempts to move outside of the physical domain, nothing 
    happens (i.e., a "bounce-back boundary condition" is enforced).
    """

    
    SUSCEPTIBLE = DiseaseStatus.SUSCEPTIBLE

    def __init__(self,population_size, no_init_infected=4,nx=10,ny=10, q=0.9):
        """
        :param population_size: The total number of people (N).
        :param no_init_infected: The number of infected people at t=0.
        :param nx: The number of lattice nodes in the x-direction 
        :param ny: The number of lattice nodes in the y-direction.
        :param q: The probability of infection (0 <= q <= 1).
        """
        self.N_ = population_size
        self.I0_= no_init_infected
        self.nx_= nx
        self.ny_= ny
        self.infection_probability_ = q
        self.State_ = np.full(self.N_, DiseaseStatus.SUSCEPTIBLE)
        self.State_[0:self.I0_] = DiseaseStatus.INFECTIOUS
       
        self.Walkers_ = np.random.randint(0, [self.nx_, self.ny_], size=(self.N_, 2))
        self.Walkers_Old_ = self.Walkers_.copy()

    def is_in_invalid_position(self, walker):
        x_check = any([walker[0] >= self.nx_ , walker[0] < 0])
        y_check =any([walker[1] >=self.ny_ , walker[1] <0])
        return x_check or y_check

    def move_walkers(self):
        self.Walkers_Old_ = self.Walkers_.copy()
        random_step_index =np.random.randint(0,4, size =(self.N_))
        next_steps = np.array([[0,1], [1,0], [0,-1], [-1,0]])
        self.Walkers_ += next_steps[random_step_index]
       
        for walker_index in range(self.N_):
            if self.is_in_invalid_position(self.Walkers_[walker_index]):
                self.Walkers_[walker_index] = self.Walkers_Old_[walker_index]   
    
    
    def coordinates_to_real_line(self, array):
        """Function that maps coordinates in strictly positive (x,y) grid, to real line isomorphically
        used in collision detection"""
        if array[0] >= self.nx_ or array[1] >= self.nx_:
            raise Exception('error, out of range') 
        return (self.nx_)*array[0]+array[1]
    
    def real_line_to_coordinates(self,value):
        """ Inverse map of coordinates to real line
        """
        if value == 0:
            return [0,0]
        else:
            return [int((value-(value % self.nx_))/self.nx_),int(value % self.nx_)] 


    def confusing_intersector(array1, array2):
        """Somewhat confsuing function that: Takes in two (n, 2) arrays
        e.g arrays of the type array1 = [[1,2], [2,1], ...]
        finds (identical) repeated elements in between the two arrays ans gives out 
        index of location of this element in both arrays.

        """
        x1v,x2v = np.meshgrid(array2[:,0],array1[:,0])
        y1v,y2v = np.meshgrid(array2[:,1],array1[:,1])

        x_diffmat = x1v-x2v 
        y_diffmat = y1v -y2v

        plusmat = x1v-x2v +y1v -y2v
        subtrmat = x1v-x2v -(y1v -y2v)

        plusmat_array1_zero = np.where(plusmat ==0)
        subtrmat_array1_zero = np.where(subtrmat ==0)

        array_1_all_intersect = reduce(np.intersect1d,(np.where(x_diffmat== 0)[0],np.where(y_diffmat ==0)[0],plusmat_array1_zero[0],subtrmat_array1_zero[0]))
        array_2_all_intersect = reduce(np.intersect1d,(np.where(x_diffmat== 0)[1],np.where(y_diffmat ==0)[1],plusmat_array1_zero[1],subtrmat_array1_zero[1]))
        
        return (array_1_all_intersect, array_2_all_intersect)
    
    def collision(self):
        infected_indices = np.where(self.State_ == DiseaseStatus.INFECTIOUS)[0]
        infected_positions = self.Walkers_[infected_indices]
        infected_positions_1d =np.array([self.coordinates_to_real_line(point) for point in infected_positions])
        
        susceptible_indices = np.where(self.State_== DiseaseStatus.SUSCEPTIBLE)[0]
        susceptible_positions = self.Walkers_[susceptible_indices]
        
        susceptible_positions_1d =np.array([self.coordinates_to_real_line(point) for point in susceptible_positions])
        collisions_on_real_line = np.intersect1d(infected_positions_1d, susceptible_positions_1d)

        indeces_for_may_be_infected = np.where(susceptible_positions_1d == collisions_on_real_line)
        

        #collision_coordinates =np.array([self.real_line_to_coordinates(val) for val in collisions_on_real_line])
        susc_to_infected_index =[]
        for infected in infected_positions_1d:
            susc_to_infected_index.append(np.where(susceptible_positions_1d == infected)[0])
         
        for index in susc_to_infected_index:
            self.State_[index] = DiseaseStatus.INFECTIOUS


    def topdown():
        pass


       
        
                





In [470]:
test_instance = RandomWalkEpidemicSimulator(population_size=20)



In [478]:
for i in range (0,50):
    test_instance.move_walkers()
    test_instance.collision()
test_instance.State_       




array([<DiseaseStatus.INFECTIOUS: 1>, <DiseaseStatus.INFECTIOUS: 1>,
       <DiseaseStatus.INFECTIOUS: 1>, <DiseaseStatus.INFECTIOUS: 1>,
       <DiseaseStatus.INFECTIOUS: 1>, <DiseaseStatus.INFECTIOUS: 1>,
       <DiseaseStatus.INFECTIOUS: 1>, <DiseaseStatus.INFECTIOUS: 1>,
       <DiseaseStatus.INFECTIOUS: 1>, <DiseaseStatus.INFECTIOUS: 1>,
       <DiseaseStatus.INFECTIOUS: 1>, <DiseaseStatus.SUSCEPTIBLE: 0>,
       <DiseaseStatus.INFECTIOUS: 1>, <DiseaseStatus.SUSCEPTIBLE: 0>,
       <DiseaseStatus.SUSCEPTIBLE: 0>, <DiseaseStatus.SUSCEPTIBLE: 0>,
       <DiseaseStatus.SUSCEPTIBLE: 0>, <DiseaseStatus.SUSCEPTIBLE: 0>,
       <DiseaseStatus.SUSCEPTIBLE: 0>, <DiseaseStatus.SUSCEPTIBLE: 0>],
      dtype=object)

In [581]:
testarray1 = np.array([[1,2],
                       [5,0],
                       [0,6]])


testarray2 = np.array([[6,5],
                       [1,3],
                       [6,6],
                       [1,3]])

x1v,x2v = np.meshgrid(testarray1[:,0],testarray2[:,0])
y1v,y2v = np.meshgrid(testarray1[:,1], testarray2[:,1])

#print(x1v)
# print(x2v)
print(x1v-x2v)

# print(y1v)
# print(y2v)
print(y1v-y2v)

print(x1v-x2v +y1v -y2v)
print(x1v-x2v -(y1v -y2v))

[[-5 -1 -6]
 [ 0  4 -1]
 [-5 -1 -6]
 [ 0  4 -1]]
[[-3 -5  1]
 [-1 -3  3]
 [-4 -6  0]
 [-1 -3  3]]
[[-8 -6 -5]
 [-1  1  2]
 [-9 -7 -6]
 [-1  1  2]]
[[-2  4 -7]
 [ 1  7 -4]
 [-1  5 -6]
 [ 1  7 -4]]


In [582]:
from functools import reduce

def confusing_intersector(array1, array2):
    x1v,x2v = np.meshgrid(array2[:,0],array1[:,0])
    y1v,y2v = np.meshgrid(array2[:,1],array1[:,1])

    x_diffmat = x1v-x2v 
    y_diffmat = y1v -y2v

    plusmat = x1v-x2v +y1v -y2v
    subtrmat = x1v-x2v -(y1v -y2v)

    plusmat_array1_zero = np.where(plusmat ==0)
    subtrmat_array1_zero = np.where(subtrmat ==0)

    array_1_all_intersect = reduce(np.intersect1d,(np.where(x_diffmat== 0)[0],np.where(y_diffmat ==0)[0],plusmat_array1_zero[0],subtrmat_array1_zero[0]))
    array_2_all_intersect = reduce(np.intersect1d,(np.where(x_diffmat== 0)[1],np.where(y_diffmat ==0)[1],plusmat_array1_zero[1],subtrmat_array1_zero[1]))
    
    return (array_1_all_intersect, array_2_all_intersect)


lets_try_this_again(testarray1,testarray2)

(array([], dtype=int64), array([], dtype=int64))

# Exercise 1

<ol>
<li> <code>reset_model</code>: set correct values for all model parameters at \( t=0 \). This includes assigning a starting location for each walker.</li>
<li> <code>move_walkers</code>: Move all walkers a single step.</li>
<li> <code>revert_illegal_moves</code>: For each move that was actually illegal, go back. To be able to do so, you must have temporarily stored the old positions.</li>
<li> <code>collide</code>: check for new infections, whether previously infected individuals become contagious (if including a latent period), if an infected person becomes immune or dies etc. Update the current state accordingly.</li>
<li> <code>plot_current_state</code>: visualize walker positions, and/or selected summary statistics (total number of infectious, recovered, dead, etc.). This is very useful for debugging. However, you should not automatically plot every time step, only selected ones specified by the user.</li>
<li> <code>save_current_state</code>: Save (selected) information about the current state for later retrieval (also useful for debugging, and needed for doing statistics). At the very least, you need to record how many people there are in each "disease compartment" at each time step.</li>
<li> <code>simulate</code>: Top-level function that organizes a full simulation from start to finish (using the other steps).</li>
</ol>

It is recommended that you code all of these steps as separate Python functions, and moreover that you place them all within a single class. You can of course use different function names if you want, but they should be meaningful. Appendix A contains example code for the beginning of a class implementation.